In [2]:
import torch
import numpy as np
import pickle
from tqdm import tqdm
tqdm.pandas()

In [3]:
def compute_paragraph_similarities_gpu(df, split_df, emb_dim=768, batch_size=1024):
    text_to_full_emb = dict(zip(df['title'], df['full_text_emb']))
    split_df['full_text_emb'] = split_df['title'].map(text_to_full_emb)
    valid_mask = split_df['full_text_emb'].notnull() & split_df['paragraph_text_emb'].notnull()
    valid_split = split_df[valid_mask].copy()
    n = len(valid_split)
    similarities = np.empty(n, dtype=np.float32)

    para_embs_all = np.stack(valid_split['paragraph_text_emb'].values)
    full_embs_all = np.stack(valid_split['full_text_emb'].values)

    for i in tqdm(range(0, n, batch_size), desc="Cosine similarity (GPU)"):
        para_embs = torch.tensor(para_embs_all[i:i+batch_size], dtype=torch.float32).cuda()
        full_embs = torch.tensor(full_embs_all[i:i+batch_size], dtype=torch.float32).cuda()
        para_embs = torch.nn.functional.normalize(para_embs, dim=1)
        full_embs = torch.nn.functional.normalize(full_embs, dim=1)
        sim = (para_embs * full_embs).sum(dim=1).cpu().numpy()
        similarities[i:i+len(sim)] = sim

    split_df.loc[valid_mask, 'similarity'] = similarities
    split_df.loc[~valid_mask, 'similarity'] = np.nan
    return split_df


In [4]:
with open('../data/train_paragraph_emb.pkl', 'rb') as f:
    split_df = pickle.load(f)

with open('../data/train_full_emb.pkl', 'rb') as f:
    df = pickle.load(f)

In [7]:
df.head()

,title,full_text,generated,full_text_emb
0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,"[-0.1765688, -0.3683777, -0.4870837, 0.1908601..."
1,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,"[0.06949594, 0.202716, -0.563547, 0.11500987, ..."
2,엘자스-로트링겐 평의회 공화국,엘자스-로트링겐 평의회 공화국은 1차대전 말기 독일 혁명 와중에 엘자스-로트링겐에서...,0,"[-0.009081549, 0.2680534, -0.48004606, -0.0524..."
3,윌리엄 페니 브룩스,"윌리엄 페니 브룩스(, 1809년 8월 13일 ~ 1895년 12월 11일)는 잉글...",0,"[-0.023021482, -0.47313088, -0.4042247, -0.083..."
4,미그로,"미그로 또는 미그로스(""Migros"")는 스위스 최대 소매 회사이자, 최대 슈퍼마켓...",0,"[-0.17965417, -0.6143994, -0.48700038, 0.04297..."


In [5]:
split_df.head()

,title,paragraph_index,paragraph_text,generated,paragraph_text_emb
0,카호올라웨섬,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,"[0.122686796, -0.6444619, -0.30836937, -0.0368..."
1,카호올라웨섬,1,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...,0,"[0.016254716, -0.42354694, -0.42748046, -0.128..."
2,카호올라웨섬,2,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...,0,"[0.012692592, -0.52005637, -0.40349406, -0.184..."
3,카호올라웨섬,3,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...,0,"[-0.0023940112, -0.51140016, -0.3829431, -0.14..."
4,카호올라웨섬,4,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...,0,"[0.01739738, -0.45859525, -0.42682236, -0.2070..."


In [6]:
split_df=compute_paragraph_similarities_gpu(df, split_df)

Cosine similarity (GPU): 100%|██████████| 1198/1198 [00:02<00:00, 481.90it/s] 


In [9]:
split_df.head(10)

,title,paragraph_index,paragraph_text,generated,paragraph_text_emb,full_text_emb,similarity
0,카호올라웨섬,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,"[0.122686796, -0.6444619, -0.30836937, -0.0368...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.799216
1,카호올라웨섬,1,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...,0,"[0.016254716, -0.42354694, -0.42748046, -0.128...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.815344
2,카호올라웨섬,2,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...,0,"[0.012692592, -0.52005637, -0.40349406, -0.184...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.806726
3,카호올라웨섬,3,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...,0,"[-0.0023940112, -0.51140016, -0.3829431, -0.14...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.801702
4,카호올라웨섬,4,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...,0,"[0.01739738, -0.45859525, -0.42682236, -0.2070...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.798503
5,카호올라웨섬,5,1941년 12월 7일에 일어난 일본 제국 해군의 진주만 공격을 계기로 카호올라웨섬...,0,"[0.10079401, -0.49370286, -0.33523205, -0.2155...","[-0.1765688, -0.3683777, -0.4870837, 0.1908601...",0.793214
6,청색거성,0,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0,"[0.12092004, 0.021827826, -0.5942758, -0.07440...","[0.06949594, 0.202716, -0.563547, 0.11500987, ...",0.882844
7,청색거성,1,"용어는 각자 다른 진화 단계에 있는 여러 가지 별에 적용되는데, 이들 모두 주계열에...",0,"[0.085140295, -0.10585993, -0.5462075, 0.03112...","[0.06949594, 0.202716, -0.563547, 0.11500987, ...",0.833844
8,청색거성,2,"청색거성이라는 명칭은 종종 매우 크고 뜨거운 주계열성과 같이, 다른 무겁고 밝은 별...",0,"[0.14214034, 0.068103544, -0.56389374, -0.0624...","[0.06949594, 0.202716, -0.563547, 0.11500987, ...",0.872846
9,청색거성,3,청색거성은 엄격히 정의된 단어가 아니어서 서로 다른 다양한 유형의 별에 폭넓게 사용...,0,"[0.05902527, 0.028855544, -0.51923007, 0.04045...","[0.06949594, 0.202716, -0.563547, 0.11500987, ...",0.803885


In [8]:
split_df.to_pickle("../data/train_paragraph_emb_sim.pkl")